## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import Tensor

from Datasets.Dataset_Loaders.breast_cancer_dataset_loader import prepare_wisc_breast_cancer_dataset
from Datasets.Dataset_Loaders.kbit_parity_datset_loader import generate_k_bit_parity_dataset
from Utils.neural_network import initialize_FC_neural_net, add_neuron_to_network, accuracy, forward_pass
from Utils.newton_update import compute_minimizer

# Globals
USE_KBIT_DATASET = True

## Dataset Loading Examples

In [2]:
# k-bit parity dataset generation
if USE_KBIT_DATASET:
    X, y = generate_k_bit_parity_dataset(k=4)
    print(f"X shape: {X.shape}")
    print(f"Y shape: {y.shape}")

X shape: torch.Size([16, 4])
Y shape: torch.Size([16, 1])


In [3]:
# Breast cancer dataset loading
if not USE_KBIT_DATASET:
    X, y = prepare_wisc_breast_cancer_dataset(convert_to_tensor=True)
    print(f"X shape: {X.shape}")
    print(f"Y shape: {y.shape}")

## Feedforward Neural Network Construction Algorithm

In [8]:
def run_FNNCA(X: Tensor, y_true: Tensor, 
              acc_threshold: float, max_hidden_units: int):
    """ Description: Performs Feedforward Neural Network Construction 
                    Algorithm with BFGS/SR1 update method.
        Args:
            X (Tensor): Input tensor of feature vectors
            y_true (Tensor): Input tensor of true labels
            acc_threshold (float): Accuray threshold for the given
                                classification problem
            max_hidden_units (int): Max number of hidden units that can be added
                                    before termination if acc_threshold not reached
    """
    # Initialize neural network with 2 initial hidden units
    n_hidden = 2
    n_feats = X.shape[1]
    W_init = initialize_FC_neural_net(n_feats, n_hidden)
    accuracy = 0
    while True:
        W = compute_minimizer(W_init, X, y_true, n_hidden)
        _, _, y_hat = forward_pass(X, W, n_feats, n_hidden)
        accuracy = accuracy(y_hat, y_true)
        print(f"Number of hidden units: {n_hidden} | Accuracy after optimization: {accuracy}")
        # If accuracy is sufficient or max network size reached, terminate
        if (accuracy < acc_threshold and n_hidden <= max_hidden_units):
            break
        # Otherwise, add another neuron and reoptimize
        else: 
            W, n_hidden = add_neuron_to_network(W, n_feats, 
                                    n_output_neurons=1, n_hidden=n_hidden)
    return W

## Reproduce Study Results

In [11]:
## kbit parity problem
X, y_true = generate_k_bit_parity_dataset(k=4)
W_final = run_FNNCA(X, y_true, acc_threshold=60, max_hidden_units=10)


Iteration 0 | grad norm: 9.20e-01:   0%|          | 0/1000 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'function' and 'Tensor'